In [1]:
import psycopg2
import pandas as pd

pd.set_option('display.max_columns', None)

#### Query data from PostgreSQL table, save to csv

In [105]:
conn = psycopg2.connect(host="localhost", dbname='cs', user='postgres', password='123456', port=5432)
cur = conn.cursor()

query = """SELECT * FROM side_dataset where id>=684"""
cur.execute(query)

rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=columns)

cur.close()
conn.close()

df.to_csv('df_642.csv')

#### Begin

In [44]:
data = pd.read_csv('df_642.csv')
#First 4 rows are not needed
df = data.iloc[:,5:]

In [45]:
df.head()

,t1_winstreak,t2_winstreak,t1_h2h,t2_h2h,t1_ranking,t2_ranking,t1_pluses,t2_pluses,t1_minuses,t2_minuses,t1_coef,t2_coef,t1_rating,t2_rating,t1_event_rating,t2_event_rating,t1_num_maps,t2_num_maps,t1_avg_lost,t2_avg_lost,t1_avg_won,t2_avg_won,t1_rounds_lost,t1_rounds_won,t1_fp,t1_fp_percent,t1_winrate,t1_played,t1_map_winstreak,t1_map_losestreak,t1_5v4,t1_4v5,t1_pistol,t2_rounds_lost,t2_rounds_won,t2_fp,t2_fp_percent,t2_winrate,t2_played,t2_map_winstreak,t2_map_losestreak,t2_5v4,t2_4v5,t2_pistol,result
0,3.0,0.0,0.0,0.0,15.0,16.0,3.0,1.0,1.0,1.0,1.698462,2.124615,5.53,5.31,5.53,5.31,22.0,42.0,8.21,7.35,11.12,7.00,8.5,0.00,False,0.03,1.00,3.0,0.0,0.0,0.744,0.476,0.667,6.50,4.75,False,0.18,0.50,8.0,0.0,0.0,0.680,0.270,0.438,True
1,0.0,0.0,0.0,0.0,31.0,13.0,0.0,4.0,1.0,0.0,2.754118,1.447059,5.62,5.53,5.62,5.53,11.0,35.0,5.71,7.50,7.25,10.35,9.0,5.00,False,0.07,0.75,4.0,0.0,0.0,0.750,0.378,0.750,7.20,17.33,False,0.21,0.62,8.0,0.0,0.0,0.743,0.330,0.625,True
2,0.0,0.0,0.0,0.0,8.0,118.0,4.0,0.0,0.0,1.0,1.134667,5.431334,5.34,4.97,5.34,4.97,38.0,16.0,6.84,8.62,8.37,10.25,4.0,6.67,False,0.11,0.40,5.0,0.0,0.0,0.702,0.265,0.500,7.00,12.33,False,0.00,0.40,5.0,0.0,0.0,0.737,0.227,0.500,False
3,0.0,0.0,1.0,6.0,48.0,18.0,0.0,3.0,3.0,0.0,2.255294,1.613529,5.08,5.52,5.08,5.52,32.0,62.0,9.65,8.46,9.27,9.48,10.2,17.00,False,0.15,0.71,7.0,0.0,0.0,0.000,0.000,0.000,8.38,11.40,True,0.49,0.57,14.0,0.0,0.0,0.764,0.272,0.500,False
4,0.0,0.0,2.0,0.0,14.0,12.0,3.0,2.0,1.0,1.0,1.320588,3.277647,5.60,4.83,5.60,4.83,31.0,47.0,8.19,8.08,10.90,8.43,10.0,11.50,False,0.18,0.56,9.0,0.0,0.0,0.752,0.310,0.556,8.25,8.00,True,0.27,0.56,9.0,0.0,0.0,0.682,0.278,0.333,True


#### While parsing the data, I was storing this particular stat (average rounds lost(won) on a map) as 0's in case of missing data. So, I convert them to NULL, to deal with them further. Also deelting the rows with incorrect data

In [46]:
df.loc[(df['t1_rounds_lost']>0) & (df['t1_rounds_won'] == 0), 't1_rounds_won'] = None
df.loc[(df['t1_rounds_lost']==0) & (df['t1_rounds_won'] > 0), 't1_rounds_lost'] = None
df.loc[(df['t2_rounds_lost']==0) & (df['t2_rounds_won'] > 0), 't2_rounds_lost'] = None
df.loc[(df['t2_rounds_lost']>0) & (df['t2_rounds_won'] == 0), 't2_rounds_won'] = None

df.drop(df[(df['t2_5v4']==0) & (df['t2_played']>0)].index, axis=0, inplace=True)
df.drop(df[(df['t1_5v4']==0) & (df['t1_played']>0)].index, axis=0, inplace=True)

#### Try filling NULL values with 13's

In [47]:
df['t1_rounds_lost'].fillna(13, inplace=True)
df['t1_rounds_won'].fillna(13, inplace=True)
df['t2_rounds_lost'].fillna(13, inplace=True)
df['t2_rounds_won'].fillna(13, inplace=True)

#### The differences between team stats will be the predicting features, so creating the features

In [48]:
cols_name = [df.columns[i]+'_dif' for i in range(0,len(df.columns[:22]),2)] + [df.columns[i]+'_dif' for i in range(22, 24)] + [df.columns[i]+'_dif' for i in range(25, 33)]

dataset = pd.DataFrame(columns=cols_name)

dataset = pd.DataFrame(columns=cols_name)
for index, name in enumerate(cols_name[:11]):
    dataset[name] = df[df.columns[index*2]] - df[df.columns[index*2+1]]

for index, name in enumerate(cols_name[11:13]):
    dataset[name] = df[df.columns[22+index]] - df[df.columns[33+index]]

for index, name in enumerate(cols_name[13:]):
    dataset[name] = df[df.columns[25+index]] - df[df.columns[36+index]]

dataset['t1_fp'] = df['t1_fp']
dataset['t2_fp'] = df['t2_fp']
dataset['result'] = df['result']

In [49]:
dataset.head()

,t1_winstreak_dif,t1_h2h_dif,t1_ranking_dif,t1_pluses_dif,t1_minuses_dif,t1_coef_dif,t1_rating_dif,t1_event_rating_dif,t1_num_maps_dif,t1_avg_lost_dif,t1_avg_won_dif,t1_rounds_lost_dif,t1_rounds_won_dif,t1_fp_percent_dif,t1_winrate_dif,t1_played_dif,t1_map_winstreak_dif,t1_map_losestreak_dif,t1_5v4_dif,t1_4v5_dif,t1_pistol_dif,t1_fp,t2_fp,result
0,3.0,0.0,-1.0,2.0,0.0,-0.426154,0.22,0.22,-20.0,0.86,4.12,2.00,8.25,-0.15,0.50,-5.0,0.0,0.0,0.064,0.206,0.229,False,False,True
1,0.0,0.0,18.0,-4.0,1.0,1.307059,0.09,0.09,-24.0,-1.79,-3.10,1.80,-12.33,-0.14,0.13,-4.0,0.0,0.0,0.007,0.048,0.125,False,False,True
2,0.0,0.0,-110.0,4.0,-1.0,-4.296667,0.37,0.37,22.0,-1.78,-1.88,-3.00,-5.66,0.11,0.00,0.0,0.0,0.0,-0.035,0.038,0.000,False,False,False
4,0.0,2.0,2.0,1.0,0.0,-1.957059,0.77,0.77,-16.0,0.11,2.47,1.75,3.50,-0.09,0.00,0.0,0.0,0.0,0.070,0.032,0.223,False,True,True
5,-3.0,0.0,-18.0,2.0,-2.0,-1.250588,-0.76,-0.76,-36.0,1.54,-2.51,4.83,0.00,0.00,-0.50,-4.0,0.0,0.0,-0.055,-0.221,-0.187,False,False,True
